In [51]:
from moz_sql_parser import parse
import re

In [82]:
query ="ALTER    TABLE Cal_pd DROP COLUMN MARAVAIL ,  MARSOLD ,  MARREV ,  MARAVAILLY ,  MARSOLDLY ,  MARREVLY ,  MKTAVAIL ,  MKTSOLD ,  MKTREV ,  MKTAVAILLY ,  MKTSOLDLY ,  MKTREVLY ;"
query = query.lower()



## Alter


In [68]:
def alter_handler(query):
    if query.startswith("alter"):
        if "drop" in query:
            # finding the table name:
            start_table = 'table'
            end_table = 'drop'
            table_name= query[query.find(start_table)+len(start_table):query.rfind(end_table)].strip()
        
            #finding the columns
            start_col = 'column'
            end_col = ';'
            drop_columns_list= query[query.find(start_col)+len(start_col):query.rfind(end_col)].strip().split(",")
            clean_drop_col_list=[i.strip() for i in drop_columns_list]
            if "." in table_name:
                table_name=table_name.split(".")[-1]
            else:
                table_name=table_name
            final_alter_df= table_name+".drop(columns="+str(clean_drop_col_list)+")"
            return final_alter_df
        
        else:
            pass
    else:
        pass

In [69]:
alter_handler(query)

"cal_pd.drop(columns=['maravail', 'marsold', 'marrev', 'maravailly', 'marsoldly', 'marrevly', 'mktavail', 'mktsold', 'mktrev', 'mktavailly', 'mktsoldly', 'mktrevly'])"

## Truncate

In [74]:
def truncate_table(query):
    if query.startswith("truncate"):
        start_table = 'table'
        end_table = ')'
        table_name= query[query.find(start_table)+len(start_table):query.rfind(end_table)].strip()
        if "." in table_name:
            table_name=table_name.split(".")[-1]
        else:
            table_name=table_name
        final_df= table_name+"="+table_name+".truncate(before=-1, after=-1)"
        return final_df
    else:
        pass

In [77]:
query = "TRUNCATE Table AW_SLS_PRD_AM.AP_COMBINED_MISW_&CURYR ) by odbc ;"
query = query.lower()
truncate_table(query)

'ap_combined_misw_&curyr=ap_combined_misw_&curyr.truncate(before=-1, after=-1)'

In [66]:
#reference
# df.drop(columns=["day","city"])

# df=df.truncate(before=-1, after=-1)

# if query.startswith("alter"):
#     if "drop" in query:
#         # finding the table name:
#         start_table = 'table'
#         end_table = 'drop'
#         table_name= query[query.find(start_table)+len(start_table):query.rfind(end_table)].strip()
        
#         #finding the columns
#         start_col = 'column'
#         end_col = ';'
#         drop_columns_list= query[query.find(start_col)+len(start_col):query.rfind(end_col)].strip().split(",")
#         clean_drop_col_list=[i.strip() for i in drop_columns_list]
#         if "." in table_name:
#             table_name=table_name.split(".")[-1]
#         else:
#             table_name=table_name
#         final_alter_df= table_name+".drop(columns="+str(clean_drop_col_list)+")"
#         print(final_alter_df)
        
#     else:
#         pass
# else:
#     pass
        
        
# query = "TRUNCATE Table AW_SLS_PRD_AM.AP_COMBINED_MISW_&CURYR ) by odbc ;"
# query = query.lower()
# if query.startswith("truncate"):
#     start_table = 'table'
#     end_table = ')'
#     table_name= query[query.find(start_table)+len(start_table):query.rfind(end_table)].strip()
#     if "." in table_name:
#         table_name=table_name.split(".")[-1]
#     else:
#         table_name=table_name
#     table_name= table_name+".truncate(before=-1, after=-1)"
#     print(table_name)
# else:
#     pass
        

In [170]:
query =""" SELECT 
 ACTUALS*-1 AS ACTUALS ,
 BUDGET*-1 AS BUDGET , 
 SUBSTRING( _NAME_,2,2 ) AS name , 
 CAST( name,int) AS monno 
 FROM SLCT_SRVC_CATERING_TRANS  
 WHERE ACTUALS<>0 AND BUDGET<>0"""

In [171]:
query_dict = parse(query.lower())
query_dict['select']

[{'value': {'mul': ['actuals', -1]}, 'name': 'actuals'},
 {'value': {'mul': ['budget', -1]}, 'name': 'budget'},
 {'value': {'substring': ['_name_', 2, 2]}, 'name': 'name'},
 {'value': {'cast': ['name', 'int']}, 'name': 'monno'}]

In [168]:
def intermediate_select_dict(select_list):
    column_list = []
    for column_dict in select_list:
        if column_dict=="*":
            pass
        elif type(column_dict['value'])== str:
            if '.' in column_dict['value']:
                column_value = column_dict['value'].split('.')[1]
                column_table = column_dict['value'].split('.')[0]
                try:
                    alias=column_dict['name']
                except:
                    alias=""
                collsttmp={"base_col":column_value,"udf":"","Alias":alias}
                column_list.append(collsttmp)
            else:
                column_value = column_dict['value']
                column_table=""
                try:
                    alias=column_dict['name']
                except:
                    alias=""
                collsttmp={"base_col":column_value,"udf":"","Alias":alias}
                column_list.append(collsttmp)
            
            
        elif type(column_dict['value'])== dict:
            if '.' in column_dict['value']:
                column_value = column_dict['value'].split('.')[1]
                column_table = column_dict['value'].split('.')[0]
                try:
                    alias=column_dict['name']
                except:
                    alias=""
                colsttmp={"base_col":column_value, "Table":column_table,"Alias":alias}
                column_list.append(colsttmp)
            else:
                column_value=column_dict['value']
                final_col=[]
                for k,v in column_dict['value'].items():
                    if k=="case":
                        pass
                    else:
                        udf=k
                        cols=v
                        if type(cols)==str:
                            if '.' in cols:
                                col_name =cols.split('.')
                                final_col.append(col_name[1])
                            else:
                                final_col.append(cols)
                        elif type(cols)==dict:
                            for k,v in cols.items(): ###########needs to be coded
                                udf=udf+","+k
                                cols=v
                                for i in cols:
                                    final_col.append(i)
                                
                        else:
                            for i in cols:
                                if type(i)==str:
                                    if '.' in i:
                                        col_name= i.split('.')[1]
                                        final_col.append(col_name)
                                    else:
                                        final_col.append(i)
                        
                                elif type(i)==int:
                                    final_col.append(i)
                    
                                elif type(i)==dict: ## here adjustments needs to be done
                                    new_dict=i
                                    for k,v in new_dict.items():
                                        extra_udf=k
                                        udf=udf+","+extra_udf
                                        cols=v
                                        if type(cols)==list:  ## for list
                                            for i in cols:
                                                if '.' in i:
                                                    part1=i.split('.')[0] 
                                                    part2=i.split('.')[1]
                                                    final_col.append(part2)
                                                else:
                                                    final_col.append(i)
                                        elif type(cols)==str:  ## for str
                                            if '.' in cols:
                                                part1=cols.split('.')[0] 
                                                part2=cols.split('.')[1]
                                                final_col.append(part2)
                                            else:
                                                final_col.append(cols)
                                        elif type(cols)==dict:  ## for dict
                                            for k,v in cols.items():
                                                third_udf=k
                                                udf =udf+","+third_udf
                                                cols=v
                                                for i in cols:
                                                    if '.' in i:
                                                        part1=i.split('.')[0] 
                                                        part2=i.split('.')[1]
                                                        final_col.append(part2)
                                                    else:
                                                        final_col.append(i)
                                        else:
                                            pass
                                            
                                else:
                                    pass
                    try:
                        alias=column_dict['name']
                    except:
                        alias=""
                    colltmp={"base_col":final_col, "udf":udf,"Alias":alias}
                    column_list.append(colltmp)
    return column_list

In [169]:
intermediate_select_dict(query_dict['select'])

[{'base_col': ['actuals', -1], 'udf': 'mul', 'Alias': 'actuals'},
 {'base_col': ['budget', -1], 'udf': 'mul', 'Alias': 'budget'},
 {'base_col': ['_name_', 2, 2], 'udf': 'substring', 'Alias': 'name'},
 {'base_col': ['name', 'int'], 'udf': 'cast', 'Alias': 'monno'},
 {'base_col': [{'concat': [{'concat': [{'concat': [{'concat': [{'concat': ['timeframe',
             {'literal': '_'}]},
           'esflag']},
         {'literal': '_'}]},
       'rmpool']},
     {'literal': '_dem_'}]},
   'yr_lbl'],
  'udf': 'compress,concat',
  'Alias': 'idvar'}]

In [ ]:
query ="""select a.marsha,
( L3_ES_GR_FW_rn/L3_ES_GR_FW_rn_ly) - 1 AS Es_GR_RN_Chg_L3 , 
( MN_ES_GR_FW_rn/MN_ES_GR_FW_rn_ly) - 1 AS Es_GR_RN_Chg_MN ,
L3_ES_GR_FW_rn / ( L3_ES_TO_WD_rn+L3_ES_TO_WE_rn) AS Es 
from azeta"""